In [260]:
# requirements.txt
# !pip install nltk
# !pip install pymystem3
# !pip install sklearn
# !pip install numpy
# !pip install tqdm

В этой задаче вам предлагается научиться по заголовку искать статью в некотором заданном множестве.

In [1]:
# !pip install corus 
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
import random
from corus import load_lenta

Для простоты возьмём первые 10000 новостей

In [136]:
path = './lenta-ru-news.csv.gz'
corpus = []
requests = []
for i, record in zip(range(10000), load_lenta(path)):
    corpus.append([i, record.text])
    requests.append((i, record.title))

Теперь необходимо реализовать класс, отвечающий за поиск новостей. Возможным подходом будет выбрать какое-нибудь векторное представление текста (bag-of-words, tf-idf, word2vec и т.п.) и метрику расстояния (косинусное, Евклидово, манхэттенское и т.п.), а потом сортировать новости по расстоянию до заголовка. Однако, вы можете реализовывать любые ваши идеи.

In [247]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
from pymystem3 import Mystem
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np
import scipy.spatial.distance as ds


class Database:
    def __init__(self, corpus):
        self.corpus = corpus.copy()
        self.prepared_corpus = []
        self.stop_words = stopwords.words("russian")
        self.stemmer = SnowballStemmer("russian")
        self.tf_idf_transformer = TfidfVectorizer()
        self.p = punctuation + '—'
        for i in tqdm(range(len(self.corpus))):
            # Lowercase
            text = self.corpus[i][1].lower()
            # Deleting punctuation
            text = ''.join([i for i in text if i not in self.p])
            # Deleting stop-words and stemming
            text = ' '.join([self.stemmer.stem(word) for word in text.split() if word not in self.stop_words])
            # Replacing '\n' at the end and saving result
            self.prepared_corpus.append(text.replace('\n', ''))
        # Transforming prepared text using TfIdf
        self.vectors = self.tf_idf_transformer.fit_transform(self.prepared_corpus)

    def find(self, request, k=10):
        """
            Этот метод должен принимать на вход текст заголовка и возвращать
            для него k самых вероятных новости.
            В качестве возвращаемого значения ожидается numpy-массив размера k, 
            содержащий id новостей в порядке уменьшения релевантности
        """
        # Preparing request such as a corpus
        text = request.lower().replace('\xa0', ' ')
        text = ''.join([i for i in text if i not in self.p])
        text = ' '.join([self.stemmer.stem(word) for word in text.split() if word not in self.stop_words])
        text = text.replace('\n', '')
        # Transforming using fitted TfIdf
        target_vector = self.tf_idf_transformer.transform([text])[0].toarray()
        dist = np.array([0.] * self.vectors.shape[0])
        # Calculating distances between vectors using euclidean distance
        for i in tqdm(range(self.vectors.shape[0])):
            dist[i] = ds.euclidean(self.vectors[i].toarray(), target_vector)
        # Retrieving k samples with lowest dist
        return np.argsort(dist)[:k]

In [248]:
%%time
database = Database(corpus)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:36<00:00, 103.73it/s]


Wall time: 1min 38s


Проверим глазами разумность ранжирования новостей на отдельном примере 

In [249]:
request_id, request_text = requests[17]
print(f'For request (id={request_id}): {request_text}')
print('Responses are:')
for i, response_id in enumerate(database.find(request_text)):
    print(f'{i}    id={response_id}\t{corpus[response_id][1]}')

  2%|█▊                                                                          | 244/10000 [00:00<00:04, 2422.31it/s]

For request (id=17): Пентагон проигнорировал обращение России по ракетному договору
Responses are:


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 2989.05it/s]

0    id=17	Пентагон проигнорировал обращение министра обороны России Сергея Шойгу обсудить ситуацию вокруг Договора о ликвидации ракет малой и средней дальности (ДРСМД) и проблемы в Сирии. Об этом заявил официальный представитель Минобороны генерал-майор Игорь Конашенков, передает «Интерфакс». По его словам, атташе по вопросам обороны при посольстве США в России несколько дней назад передал главе Пентагона Джеймсу Мэттису два обращения. Первое касалось усиления курдско-арабского конфликта на подконтрольной США территории Сирии восточнее Евфрата. Во второй ноте Шойгу предложил обсудить имеющиеся разногласия по условиям соблюдения ДРСМД. «По прошествии трех суток даже формальной реакции на данные предложения Минобороны России от американского военного ведомства не последовало», — заявил Конашенков. Он подчеркнул, что в обращении Шойгу говорилось о готовности российской стороны к открытому и предметному диалогу. «Все это говорит о нежелании американской стороны к аргументированному и проф

Теперь оценим качество ранжирования по метрике Recall@k 

In [250]:
def get_recall_at_k(targets, predictions, k):
    targets_mask = np.repeat(np.expand_dims(targets, 1), k, axis=1)
    return (predictions[:, :k] == targets_mask).sum() / len(targets)

In [230]:
test_size = 256
test_k = 20

In [257]:
test_requests = random.sample(requests, test_size)

In [258]:
%%time
targets = np.zeros(test_size, dtype=np.int32)
predictions = np.zeros((test_size, test_k), dtype=np.int32)
for i, (request_id, request_text) in enumerate(test_requests):
    targets[i] = request_id
    predictions[i] = database.find(request_text, k=test_k)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4273.94it/s]

Wall time: 10min 19s


In [259]:
for k in [1, 3, 5, 10, 20]:
    print(f'Recall@{k}:\t{get_recall_at_k(targets, predictions, k):.3f}')

Recall@1:	0.477
Recall@3:	0.703
Recall@5:	0.762
Recall@10:	0.824
Recall@20:	0.879
